# Описание гипотезы
### Проверим предположения
1. Определить, насколько обучение сокращает время прохождения этапов игры.
2. Доказать, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.
3. Проверить, насколько прозрачен процесс взаимодействия с игрой.
#### Проверять будем на данных пользователей, зарегистрированных в 2018 году (1 января 2018 - 31 декабря 2018)

Импротируем нужные библиотеки

In [122]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

Загружаем данные

In [203]:
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')

## Производим предобработку данных

Выделяем пользователей зарегистрирванных в 2018 году и загружаем данные по ним в отдельные датафреймы

In [204]:
# Создаем фильтр по пользователям, зарегистрировавшихся в 2018 году
mask = (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] < '2019-01-01') & (events_df['event_type'] == 'registration')

regist_2018 = events_df[mask]['user_id'].to_list() # список пользователей зарегистрировавшихся в 2018 году 

events_2018 = events_df[events_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице events
purchase_2018 = purchase_df[purchase_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице purchase


Преобразуем даты в полученых датафреймах

In [205]:
purchase_2018['event_datetime'] = pd.to_datetime(purchase_2018['event_datetime'])
events_2018['start_time'] = pd.to_datetime(events_2018['start_time'])

Перед объединением датафреймов, переименуем колонки идентификаторов

In [206]:
events_2018 = events_2018.rename(columns={'id':'event_id'})
purchase_2018 = purchase_2018.rename(columns={'id':'purchase_id'})
purchase_2018['start_time'] = purchase_2018['event_datetime']
purchase_2018['event_type'] = 'purchase'

Сделаем объединенный датафрейм из событий и оплат

In [207]:
total_events_df = pd.concat([events_2018,purchase_2018],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

### Проверим 1-ю гипотезу
* Насколько обучение сокращает время прохождения этапов игры

Сформируем группы пользователей, которые проходили обучение и которые не проходили

In [196]:
# Пользователи которые прошли обучение
users_tutorial_finish = total_events_df[total_events_df['event_type'] == 'tutorial_finish']['user_id'].unique()
print('Количество пользователей завершивших обучение: {}'.format(len(users_tutorial_finish)))
# Пользователи которые не проходили обучение
users_not_tutorial = total_events_df[~(total_events_df['user_id'].isin(users_tutorial_finish))]['user_id'].unique()
print('Количество пользователей не проходивших обучение: {}'.format(len(users_not_tutorial)))

Количество пользователей завершивших обучение: 10250
Количество пользователей не проходивших обучение: 9676


Разобьем пользователей на группы

In [209]:
total_events_df['groups'] = total_events_df.apply(lambda x: 'С обучением' if x['user_id'] in users_tutorial_finish else 'Без обучения', axis=1)

Сформируем сводную таблицу прохождения этапов игры

In [177]:
# оставляем только интерисующие нас эвенты
#users_etap = total_events_df[(total_events_df['event_type'] == 'pack_choice') | (total_events_df['event_type'] == 'registration')]

In [231]:
users_etap = total_events_df.pivot_table(
    values='start_time',
    index=['user_id','groups'],
    columns='event_type',
    aggfunc='min'
)

Вычислим разницу во времени прохождения этапов игры

In [220]:
users_etap['level_diff'] = users_etap['level_choice'] - users_etap['registration']
users_etap['pack_diff'] = users_etap['pack_choice'] - users_etap['level_choice']
users_etap['purchase_diff'] = users_etap['purchase'] - users_etap['pack_choice']
#users_etap['time_diff'] = (users_etap['level_diff'] + users_etap['pack_diff'] + users_etap['purchase_diff']).mean()

Вычислим среднее время прохождения по группам

In [227]:
group_avg = users_etap.groupby(by='groups')[['level_diff','pack_diff','purchase_diff']].mean()
#group_avg['time_diff'] = group_avg['time_diff'] / 1000000

In [266]:
dt = {'groups':[], 'diff':[], 'time':[]}
for col in group_avg.columns:
    for ind, gr in enumerate(group_avg[col]):
        dt['groups'].append(group_avg.index[ind])
        dt['diff'].append(col)
        dt['time'].append(gr/1000000)

groups_df = pd.DataFrame(data=dt)
    

{'groups': ['Без обучения', 'С обучением', 'Без обучения', 'С обучением', 'Без обучения', 'С обучением'], 'diff': ['level_diff', 'level_diff', 'pack_diff', 'pack_diff', 'purchase_diff', 'purchase_diff'], 'time': [Timedelta('0 days 00:00:00.028337178'), Timedelta('0 days 00:00:00.025536854'), Timedelta('0 days 00:00:00.000307739'), Timedelta('0 days 00:00:00.000318146'), Timedelta('0 days 00:00:00.339610483'), Timedelta('0 days 00:00:00.321479641')]}


Выведим график

In [272]:
fig = px.bar(
    data_frame=groups_df,
    x='groups', #group_avg.index,
    y='time',
    color='diff',
    orientation='v',
    height=500,
    width=1000,
    barmode='group',
    title='Среднее время прохождения этапов игры',
    labels={'time':'Время прохождения', 'groups':'Группы'},
    log_y=True
)
fig.update_yaxes({'type': 'date', 'tickformat': '%d %H:%M:%S'}, range=[0,40000000])
fig.show()

### Вывод по гипотезе №1
* 

In [230]:
total_events_df

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount,groups
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN,Без обучения
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN,С обучением
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN,С обучением
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN,С обучением
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN,С обучением
...,...,...,...,...,...,...,...,...,...,...
68554,NaN,purchase,NaN,2019-01-02 03:48:19,NaN,47498,18441.0,2019-01-02 03:48:19,100.0,С обучением
68555,NaN,purchase,NaN,2019-01-02 23:26:26,NaN,47647,18442.0,2019-01-02 23:26:26,150.0,С обучением
68556,NaN,purchase,NaN,2019-01-03 00:36:36,NaN,47554,18443.0,2019-01-03 00:36:36,50.0,С обучением
68557,NaN,purchase,NaN,2019-01-04 12:51:41,NaN,47742,18448.0,2019-01-04 12:51:41,50.0,С обучением


In [234]:
group_avg.columns

Index(['level_diff', 'pack_diff', 'purchase_diff'], dtype='object', name='event_type')

In [232]:
users_etap

,event_type,level_choice,pack_choice,purchase,registration,tutorial_finish,tutorial_start
user_id,groups,,,,,,
27832,Без обучения,NaT,NaT,NaT,2018-01-01 03:48:40,NaT,NaT
27833,С обучением,NaT,NaT,NaT,2018-01-01 04:07:25,2018-01-01 17:50:08,2018-01-01 17:47:40
27834,С обучением,NaT,NaT,NaT,2018-01-01 08:35:10,2018-01-01 19:48:01,2018-01-01 19:46:11
27835,С обучением,2018-01-01 20:37:22,2018-01-01 20:38:43,NaT,2018-01-01 11:54:47,2018-01-01 15:06:15,2018-01-01 15:00:51
27836,С обучением,NaT,NaT,NaT,2018-01-01 13:28:07,2018-01-01 15:42:58,2018-01-01 14:54:40
...,...,...,...,...,...,...,...
47753,С обучением,2018-12-31 22:36:19,2018-12-31 22:38:05,NaT,2018-12-31 18:58:55,2018-12-31 20:57:07,2018-12-31 20:54:56
47754,Без обучения,NaT,NaT,NaT,2018-12-31 19:14:08,NaT,NaT
47755,С обучением,2019-01-01 05:04:52,NaT,NaT,2018-12-31 21:15:14,2019-01-01 00:09:30,2019-01-01 00:08:14


In [ ]:
group_avg.columns

Index(['time_diff'], dtype='object', name='event_type')

In [ ]:
level_choise_total

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
19,80327.0,level_choice,medium,2018-01-01 20:37:22,NaN,27835,NaN,NaT,NaN
23,80331.0,level_choice,hard,2018-01-01 22:37:50,NaN,27839,NaN,NaT,NaN
36,80344.0,level_choice,medium,2018-01-02 05:18:42,NaN,27840,NaN,NaT,NaN
38,80346.0,level_choice,hard,2018-01-02 06:19:18,NaN,27845,NaN,NaT,NaN
45,80353.0,level_choice,easy,2018-01-02 08:46:03,NaN,27842,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
66932,147240.0,level_choice,medium,2018-12-31 09:59:00,NaN,47732,NaN,NaT,NaN
66937,147245.0,level_choice,medium,2018-12-31 12:24:40,NaN,47747,NaN,NaT,NaN
66948,147256.0,level_choice,medium,2018-12-31 22:36:19,NaN,47753,NaN,NaT,NaN
66956,147265.0,level_choice,easy,2019-01-01 05:04:52,NaN,47755,NaN,NaT,NaN
